# Covid-19 Forecasting using an RNN

The goal of this notebook is very simple: Generate additional features from the Covid19-global-forecasting dataset and feed it into an RNN. The RNN will take as inputs:
*     number of cases for 13 days
*     number of fatalities for 13 days

as outputs:
*    number of cases for the 14th day
*    number of fatalities for the 14th day

* V5: Submission pipeline fixed - score: 3.09681
* V6: New RNN architecture with two separate branches for each output - score: 2.25901
* V8: Add a post-processing step checking if the model's output is equal or greater the previous value - score: 2.29932
* V9: Change the MSE losses to RMSLE - score: 1.43247
* V11: Change the outputs' activation fucntions from linear to ReLU - score: 1.26594
* V12: Use a 2-week period for predictions instead of 1. Replace the SimpleRNN layers with LSTM layers - score: 1.14070
* V13: Fix bug in the cell 4 (flagged by @jeremyoudin) which made the dataset much larger due to duplicates and also created a leakage between the training and validation sets. 

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os
import tensorflow as tf
from tqdm import tqdm
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta

from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_df = gpd.read_file("/kaggle/input/covid19-global-forecasting-week-2/train.csv")
train_df["Country_Region"] = [country_name.replace("'","") for country_name in train_df["Country_Region"]]
train_df.head()

I create a new dataframe where I will only store 6-day trends for each location with the resulting numbers on the 7th day. The time periods extracted do not overlap on purpose.

In [ ]:
trend_df = pd.DataFrame(columns={"infection_trend","fatality_trend","expected_cases","expected_fatalities"})

In [ ]:
#Just getting rid of the first days to have a multiple of 14
#Makes it easier to generate the sequences
train_df = train_df.query("Date>'2020-01-22'and Date<='2020-03-18'")
days_in_sequence = 14

with tqdm(total=len(list(train_df.Country_Region.unique()))) as pbar:
    for country in train_df.Country_Region.unique():
#         import pdb; pdb.set_trace()
        for province in train_df.query("Country_Region=='"+country+"'").Province_State.unique():
            province_df = train_df.query("Country_Region=='"+country+"' and Province_State=='"+province+"'")
            for i in range(0,len(province_df)-days_in_sequence,days_in_sequence):

                infection_trend = [float(x) for x in province_df[i:i+days_in_sequence-1].ConfirmedCases.values]
                fatality_trend = [float(x) for x in province_df[i:i+days_in_sequence-1].Fatalities.values]
                expected_cases = float(province_df.iloc[i+days_in_sequence].ConfirmedCases)
                expected_fatalities = float(province_df.iloc[i+days_in_sequence].Fatalities)
                                            
                trend_df = trend_df.append({"infection_trend":infection_trend,
                                 "fatality_trend":fatality_trend,
                                 "expected_cases":expected_cases,
                                 "expected_fatalities":expected_fatalities},ignore_index=True)
        pbar.update(1)

Shuffling the dataframe to make sure we have a bit of everything in our training and validation set.

In [ ]:
trend_df["input"] = [np.asarray([trends["infection_trend"],trends["fatality_trend"]]) for idx,trends in trend_df.iterrows()]

In [ ]:
trend_df = shuffle(trend_df)
trend_df.head()

Only keeping 2000 sequences where the number of cases stays at 0, as there were way too many of these samples in our dataset.

In [ ]:
i=0
y=0
temp_df = pd.DataFrame(columns={"infection_trend","fatality_trend","expected_cases","expected_fatalities","input"})
for idx,row in trend_df.iterrows():
    if sum(row.infection_trend)>0:
        temp_df = temp_df.append(row)
    else:
        if i<100:
            temp_df = temp_df.append(row)
            i+=1
trend_df = temp_df

In [ ]:
trend_df[:20]

Splitting my dataset - 90% for training and 10% for validation

In [ ]:
sequence_length = 13
training_percentage = 0.9

In [ ]:
training_item_count = int(len(trend_df)*training_percentage)
validation_item_count = len(trend_df)-int(len(trend_df)*training_percentage)
training_df = trend_df[:training_item_count]
validation_df = trend_df[training_item_count:]

In [ ]:
X_train = np.asarray(np.reshape(np.asarray([np.asarray(x) for x in training_df["input"]]),(training_item_count,2,sequence_length))).astype(np.float32)
Y_cases_train = np.asarray([np.asarray(x) for x in training_df["expected_cases"]]).astype(np.float32)
Y_fatalities_train = np.asarray([np.asarray(x) for x in training_df["expected_fatalities"]]).astype(np.float32)

In [ ]:
X_test = np.asarray(np.reshape(np.asarray([np.asarray(x) for x in validation_df["input"]]),(validation_item_count,2,sequence_length))).astype(np.float32)
Y_cases_test = np.asarray([np.asarray(x) for x in validation_df["expected_cases"]]).astype(np.float32)
Y_fatalities_test = np.asarray([np.asarray(x) for x in validation_df["expected_fatalities"]]).astype(np.float32)

## Build the model

The model is very simple in terms of architecture. The only difference from what could traditionally be seen is that it has two outputs so we can have two different losses (one for the expected number of cases and for the expected number of fatalities).

In [ ]:

input_layer = Input(shape=(2,sequence_length))
main_rnn_layer = layers.LSTM(128, return_sequences=True, recurrent_dropout=0.2)(input_layer)

rnn_c = layers.LSTM(64)(main_rnn_layer)
rnn_f = layers.LSTM(64)(main_rnn_layer)

dense_c = layers.Dense(256)(rnn_c)
dropout_c = layers.Dropout(0.3)(dense_c)

dense_f = layers.Dense(256)(rnn_f)
dropout_f = layers.Dropout(0.3)(dense_f)

cases = layers.Dense(1, activation="relu",name="cases")(dropout_c)
fatalities = layers.Dense(1, activation="relu", name="fatalities")(dropout_f)
model = Model(input_layer, [cases,fatalities])

model.summary()

In [ ]:
callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=4, verbose=1, factor=0.7),
             EarlyStopping(monitor='val_loss', patience=20),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]
model.compile(loss=[tf.keras.losses.MeanSquaredLogarithmicError(),tf.keras.losses.MeanSquaredLogarithmicError()], optimizer='adam')

In [ ]:
history = model.fit(X_train, [Y_cases_train, Y_fatalities_train], 
          epochs = 200, 
          batch_size = 16, 
          validation_data=(X_test,  [Y_cases_test, Y_fatalities_test]), 
          callbacks=callbacks)

## Performance during training

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss over epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='best')
plt.show()

In [ ]:
plt.plot(history.history['cases_loss'])
plt.plot(history.history['val_cases_loss'])
plt.title('Loss over epochs for the number of cases')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='best')
plt.show()

In [ ]:
plt.plot(history.history['fatalities_loss'])
plt.plot(history.history['val_fatalities_loss'])
plt.title('Loss over epochs for the number of fatalities')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='best')
plt.show()

## Generate predictions using the model

We can quickly check the quality of the predictions... One thing is clear, there is room for improvement!

In [ ]:
model.load_weights("best_model.h5")

In [ ]:
predictions = model.predict(X_test)

In [ ]:
display_limit = 30
for inputs, pred_cases, exp_cases, pred_fatalities, exp_fatalities in zip(X_test,predictions[0][:display_limit], Y_cases_test[:display_limit], predictions[1][:display_limit], Y_fatalities_test[:display_limit]):
    print("================================================")
    print(inputs)
    print("Expected cases:", exp_cases, " Prediction:", pred_cases[0], "Expected fatalities:", exp_fatalities, " Prediction:", pred_fatalities[0] )

### Apply the model on this existing data

The following functions will be used to get the 6 previous days from a given date, predict the number of cases and fatalities, before iterating again. Therefore, it will use the prediction for the next day as part of the data for the one afterwards.

In [ ]:
#Will retrieve the number of cases and fatalities for the past 6 days from the given date
def build_inputs_for_date(country, province, date, df):
    start_date = date - timedelta(days=13)
    end_date = date - timedelta(days=1)
    
    str_start_date = start_date.strftime("%Y-%m-%d")
    str_end_date = end_date.strftime("%Y-%m-%d")
    df = df.query("Country_Region=='"+country+"' and Province_State=='"+province+"' and Date>='"+str_start_date+"' and Date<='"+str_end_date+"'")
    input_data = np.reshape(np.asarray([df["ConfirmedCases"],df["Fatalities"]]),(2,sequence_length)).astype(np.float32)
    
    return input_data

In [ ]:
#Take a dataframe in input, will do the predictions and return the dataframe with extra rows
#containing the predictions
def predict_for_region(country, province, df):
    begin_prediction = "2020-03-19"
    start_date = datetime.strptime(begin_prediction,"%Y-%m-%d")
    end_prediction = "2020-04-30"
    end_date = datetime.strptime(end_prediction,"%Y-%m-%d")
    
    date_list = [start_date + timedelta(days=x) for x in range((end_date-start_date).days+1)]
    for date in date_list:
        input_data = build_inputs_for_date(country, province, date, df)
        result = model.predict(np.array([input_data]))
        
        #just ensuring that the outputs is above 0
        # or higher than the previous counts
        #Get the absolute value for the number of cases
        
        result[0] = np.round(result[0])
        if result[0]<input_data[0][-1]:
            result[0]=np.array([[input_data[0][-1]]])
        
        result[1] = np.round(result[1])
        if result[1]<input_data[1][-1]:
            result[1]=np.array([[input_data[1][-1]]])
            
        df = df.append({"Country_Region":country, 
                        "Province_State":province, 
                        "Date":date.strftime("%Y-%m-%d"), 
                        "ConfirmedCases":round(result[0][0][0]),	
                        "Fatalities":round(result[1][0][0])},ignore_index=True)
    return df

In [ ]:
copy_df = train_df
with tqdm(total=len(list(copy_df.Country_Region.unique()))) as pbar:
    for country in copy_df.Country_Region.unique():
        for province in copy_df.query("Country_Region=='"+country+"'").Province_State.unique():
            copy_df = predict_for_region(country, province, copy_df)
        pbar.update(1)

Example

In [ ]:
copy_df.query("Country_Region=='France' and Province_State=='French Guiana' and Date>'2020-03-10'")

In [ ]:
test_df = gpd.read_file("/kaggle/input/covid19-global-forecasting-week-2/test.csv")
test_df.head()

Just need to do this little trick to extract the relevant date and the forecastId and add that to the submission file.

In [ ]:
submission_df = pd.DataFrame(columns=["ForecastId","ConfirmedCases","Fatalities"])
for idx, row in test_df.iterrows():
    #Had to remove single quotes because of countries like Cote D'Ivoire for example
    country_region = row.Country_Region.replace("'","").strip(" ")
    province_state = row.Province_State.replace("'","").strip(" ")
    item = copy_df.query("Country_Region=='"+country_region+"' and Province_State=='"+province_state+"' and Date=='"+row.Date+"'")
    submission_df = submission_df.append({"ForecastId":row.ForecastId,
                                          "ConfirmedCases":int(item.ConfirmedCases.values[0]),
                                          "Fatalities":int(item.Fatalities.values[0])},
                                         ignore_index=True)

In [ ]:
submission_df.sample(20)